# Build The Neural Network
Article: [here](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

Neural networks **comprise of layers/modules that perform operations on data**. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device For Training
We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let’s check to see if `torch.cuda` is available, else we continue to use the CPU.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device.')

Using cpu device.


## Define The Class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s `forward`, along with some background operations. Do not call `model.forward()` directly!

**Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class**. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


## Model Layers
Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3335,  0.5813, -0.1959, -0.3181, -0.4606, -0.3562, -0.1151, -0.2908,
          0.2991,  0.5221,  0.1389,  0.3689, -0.1902, -0.4460, -0.1666, -0.0840,
          0.2084, -0.5358,  0.4481, -0.0888],
        [ 0.6632,  0.5283, -0.4120, -0.5511, -0.2641, -0.0018, -0.2459, -0.2074,
         -0.1146,  0.7517,  0.3059,  0.1360, -0.2241, -0.3536, -0.2870, -0.0490,
          0.1496, -0.4603,  0.3524, -0.3919],
        [ 0.4752,  0.3665, -0.3735, -0.3982, -0.2287, -0.2470, -0.2425, -0.2245,
          0.0572,  1.0148, -0.0491,  0.2972, -0.1830, -0.4393, -0.2371, -0.4527,
          0.2392, -0.3158, -0.0375, -0.3758]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3335, 0.5813, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2991,
         0.5221, 0.1389, 0.3689, 0.0000, 0.0000, 0.0000, 0.0000, 0.2084, 0.0000,
         0.4481, 0.0000],
        [0.6632, 0.5283, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.7517, 0.3059, 0.1360, 0.0000, 0.0000, 0.000

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns logits - raw values in `[-infty, infty]` - which are passed to the `nn.Softmax` module. The logits are scaled to values `[0, 1]` representing the model’s predicted probabilities for each class. `dim` parameter indicates the dimension along which the values **must sum to 1**.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameter
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0135, -0.0081,  0.0356,  ..., -0.0053, -0.0336, -0.0082],
        [ 0.0211, -0.0342,  0.0108,  ...,  0.0152, -0.0243,  0.0041]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0239,  0.0288], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0426, -0.0110, -0.0279,  ...,  0.0027,  0.0144,  0.0252],
        [-0.0408, -0.0242, -0.0356,  ...,  0.0170,  0.0393,  0.0166]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | S